# GIS Visualizations with Folium and GeoPandas 

This is a demonstration of how open source visualization tools can be applied to the civic participation space. The result is a self-contained map of Washington County, Oregon, that contains easy-to-access information about a user's state house district and 'home' HDLs.


In [1]:
#SETUP

#GeoPandas depends on the fiona package, which does not play well with packages if they are installed from different conda repositories
#If installing GeoPandas with Anaconda, I recommend doing it in a fresh environment with only what is required for the visualization.
#I have confirmed that GeoPandas, fiona,(from main) and folium (from conda-forge) can be installed together without problems on Python 3.6 

import geopandas as gpd

In [2]:
#Read GeoJSON file into GeoPandas dataframe
#The oregon_lower.json originally contained ALL house district boundaries for Oregon
#I converted it to JSON at https://ogre.adc4gis.com
#Then removed any districts that do not exist in Washington County (by hand!)

#The remaining HDs are: 24,26,27,28,29,30,31,32,33,34,35,37 ---> 12 districts

gdf = gpd.read_file("oregon_lower.json")
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 10 columns):
STATEFP     12 non-null object
SLDLST      12 non-null object
AFFGEOID    12 non-null object
GEOID       12 non-null object
NAME        12 non-null object
LSAD        12 non-null object
LSY         12 non-null object
ALAND       12 non-null int64
AWATER      12 non-null int64
geometry    12 non-null object
dtypes: int64(2), object(8)
memory usage: 1.0+ KB


In [3]:
import pandas as pd 

#import HDL information, which was collected and then entered by hand
#This version for Github has been somewhat anonymized

hdl = pd.read_csv("hdl_names_anon.csv")
hdl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
SLDLST    12 non-null int64
HDL       12 non-null object
dtypes: int64(1), object(1)
memory usage: 272.0+ bytes


In order to merge the two dataframes on the key (SLDLST), both have to be have the same information and format, so the field in geopandas data frame had to be converted to int.

In [4]:
gdf.SLDLST = gdf.SLDLST.apply(lambda x: int(x)) 

In [5]:
#merge everything together to create one geopandas dataframe
everything = gdf.merge(hdl, on="SLDLST")
everything.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 12 entries, 0 to 11
Data columns (total 11 columns):
STATEFP     12 non-null object
SLDLST      12 non-null int64
AFFGEOID    12 non-null object
GEOID       12 non-null object
NAME        12 non-null object
LSAD        12 non-null object
LSY         12 non-null object
ALAND       12 non-null int64
AWATER      12 non-null int64
geometry    12 non-null object
HDL         12 non-null object
dtypes: int64(3), object(8)
memory usage: 1.1+ KB


## Visualization Procedure

While there are a lot of tutorials online for how to display a basic GeoJSON visualization correctly, many of them seem to be outdated as folium has evolved quickly.

The easiest way to actually make things work is actually to read through [the API](https://python-visualization.github.io/folium/modules.html). 

I'll walk through the logic of the code below for the following functions:
1. applying multiple layers of GeoJSON 
2. displaying tooltips bound to specific fields
3. adding plugins

First we'll apply multiple layers of GeoJSON. This will be necessary because we will want a layer to display the boundaries of Washington County and another layer to display the boundaries of the house districts within the county. Both layers are necessary because house districts do not lie neatly within county lines. 

### Special note

Despite the proliferation of government data, some of it is actually not usable by folium. **None** of the GIS data available from Oregon or Portland Metro will be displayed correctly. I had to extract the necessary counties and legislative districts from Census Bureau shapefiles instead.

I suspect it's because Oregon may be using a previous specification that is not converting correctly into JSON. Perhaps folium does not like the 'Polygon' tag when it should be "MultiPolygon"? Worth investigating later.

In [140]:
#start actual visualization with folium
#Anyway, we have a shape of Washington County, Oregon, hurray!

import folium

#to make a map in folium, simply specify where we should center with specific latitude and longitude.
#Zoom level will control how much or how little of the surroundings we see.
#the coordinates provided here are for the address of the main office of the Washington County Democrats in Hillsboro
m = folium.Map(location=[45.5231839,-122.98797], zoom_start=9.5)

In [141]:
washco = gpd.read_file('washco.json')

In [142]:
folium.features.GeoJson(washco, name='County Boundaries',control=False,
                       style_function=lambda x: {"weight":3, 'color':'black','fillColor':'white'}).add_to(m)

There are a few tutorials that talk about binding data fields to choropleths or simple GeoJSON visualizations for markers labels or tooltips. I have not been able to get them to work on the latest version of folium (Folium 0.7.0+8.g8adfb77) as many of the fields in the API have been changed. I think the programmers have simplified the command structure, as detailed below.

In [143]:
folium.features.GeoJson(everything, name="House Districts", 
                        style_function=lambda x: {"weight":2,'color':'blue','opacity':0.5,'fillColor':'blue' if x['properties']['HDL'] != 'Vacant!' else 'white','fillOpacity':'0.1'},
                        tooltip=folium.features.GeoJsonTooltip(['SLDLST','HDL'], 
                                                       aliases=['House District #', 'HDL(s)'])).add_to(m)

#We aren't using a Choropleth, so we use a simple GeoJSON visualization for the edges
#We want to specify the geopandas file that we want to use that's got the merged HDL data

#There are several types of tooltips. We specifically want the GeoJsonTooltip
#The first list shows the fields that we want to display. 
#SLDLST is the House District number, HDL shows the name of the districts.
#Note that you can use HTML formatting in tool tips
#We use aliases to prettify the field names to actual English
#Last, add to actual map

In [144]:
#Add controls to map

folium.LayerControl(autoZIndex=False, collapsed=False).add_to(m)

In [145]:
#add full screen option to map
from folium import plugins

plugins.Fullscreen(position='topleft', title='Full Screen', title_cancel='Exit Full Screen', force_separate_button=False).add_to(m)

In [146]:
m.save("prototype.html")   

In [147]:
m